# Hamper Problem - Solving with a Genetic Algorithm

This notebook details using a Genetic Algoritm to solve the "Hamper Problem" (described below). A Genetic Algorithm is a type of evolutionary algorithm modelled after natural selection that can be used to solve optimization and search problems among other use. The problem that I'll be solving was proposed as follows:

>A charity purchased some bulk packs, each pack contains 1 or more items. They want to put together as many hampers as they can, with the costs ideally of 5000, and they're all as even as possible, so minimise the sum of the absolute difference of the hamper cost to 5000 across all hampers. Design the hampers with no duplicate items.

## Problem outline

Here is my summary of the problem based on the provided information. There are some points that I would want to discuss/clarify in a real world scenario, but I have made assumptions for these for the purpose of solving the problem here.

__Goals:__
* Make multiple hampers from the items in the bulk packs
* Minimise the sum of the absolute difference from 5000 for the total cost of the hampers 
* Maximise the number of hampers (I'm assuming the above always takes priority, so this is really a moot point)

__Constraints:__
* Hampers cannot contain duplicate items
* Limited to the number of units specified in the data (i.e. can't exceed this)
* All items must be used (assumed)

__Output:__
* Number of hampers (have already calculated the optimum number to be 25)
* Sum of the absolute difference of hamper cost from 5000 across all hampers (we'll refer to this as sum cost difference from here on for brevity)
* The items in each hamper

## Imports
All libraries that are used in this notebook are imported below.

In [1]:
import json
from random import randint

from tabulate import tabulate
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random as rd
import seaborn as sns

sns.set_style("darkgrid")

## Bulk Pack Data

A CSV file containing data on the bulk packs of items was provided with the problem. This has been preprocessed to determine the total number of units for each item and the price per unit in the "Hamper Problem - Initial Processing" notebook. The preprocessed data is loaded from a CSV file below.

In [2]:
df = pd.read_csv("./data/items.csv", index_col=0)
df.head()

,item,total units,price per unit
0,Bleach,10,710.0
1,Chickpeas,10,1300.0
2,Coffee,10,2090.0
3,Flour,10,520.0
4,Lentils,10,1189.0


I'll represent this as a dictionary to make it easy to lookup item names and values.

In [3]:
item_lookup = {}
for i, row in df.iterrows():
    item_lookup[i] = row.to_dict()

## Optimum Number of Hampers and Ideal Total Cost Difference
Based on the total cost of all items and the target hamper cost of 5000, I have determined that the __optimum number of hampers is 25__. In this process I also found that __the best possible sum cost difference is 1709__. The calculation of these is detailed in the "Hamper Problem - Initial Processing" notebook. 

In [4]:
num_hampers = 25
target_cost_difference = 1709

## Creating the Genetic Algorithm

A Genetic Algorithm is a type of evolutionary algorithm that was inspired by natural selection. This algorithm will look to evolve an initial set of possible solutions (in this case randomised) to an optimised solution. It does this by "breeding" and "mutating" the best solutions (refered to as chromosomes) over multiple generations. 

The Genetic Algorithm used for this problem will take the following steps (This is visualised in the diagram below):
* __Step 1 - Initialise population__: This is done at the start of the process and provides an initial population of solutions for the algorithm to work with. In this case it will be a set of psuedo-random solutions that fit the constraits of the problem (e.g. no duplicate items in a hamper).
* __Step 2 - Fitness calculation__: The fitness calculation provides a metric to score which chromosomes are performing the best. In this case it will be the sum of the absolute difference from 5000 across all hampers.
* __Step 3 - Check if it should terminate__: Determine if the target fitness has been met or some other limit reached (e.g. total number of generations or no improvement in n generations).
* __Step 4 - Selection__: Select the fittest solutions to make the next generation
* __Step 5 - Crossover__: Combine fittest "parent" solutions to create new solutions (parents will also be in the next generation)
* __Step 6 - Mutation__: Randomly change parts of the solutions for some of the new population.
* __Repeat from Step 2__ until a termination criterum is met

<img src="./data/GAProcessDiagram.png" width=300/>

Note: Genetic Algorithms will find __an__ optimised solution not always the most optimal solution. They can potentially give different answers each time they are run. There are other approaches better suited to finding the most optimal solution (and more efficient approaches), but I wanted to play around with this kind of algorithm.

### Chromosome Structure

The genetic algorithm requires a "genetic" representation of the solution domain. These are refered to as chromosomes. The chromosomes are most commonly an array of bits, however, other structures and encoding are also used. Selecting an appropriate structure and encoding will be particularly important for this problem and will influence how we perform crossover and mutaion and how easy or difficult it will be to produce solutions within the constraints of the problem.

There are several ways in which this problem could be represented. I decided to have the chromosome represent the bulk packages and use label encoding for the hampers that the items would be assigned to. Below is an example of this for 3 packs each with 3 items distributed between 5 hampers. This strucutre ensures that the exact number of items are used (no more no less) and the sub-arrays for each bulk pack will make it easy to avoid adding duplicates to a hamper in the crossover or mutation stages.

`[[0,1,2],[3,4,1],[0,4,3]]`

The function below will be used to convert this chromosome structure to a dictionary representing the hampers

In [5]:
def chromosome_to_hampers(chromosome: list, num_hampers: int) -> list:
    # Item in hamper shouldn't be greater than the number of hampers
    if any(hamper > num_hampers - 1 for pack in chromosome for hamper in pack):
        msg = f"Hamper in chromosome is greater than num_hampers {num_hampers}"
        raise Exception(msg)

    # Empty hampers to add items into
    hampers = [[] for _ in range(num_hampers)]

    # Iterate through chromosome to put items into their hampers
    for i, pack in enumerate(chromosome):
        for assigned_hamper in pack:
            hampers[assigned_hamper].append(i)

    return hampers


hampers = chromosome_to_hampers([[0,1,2],[3,4,1],[0,4,3]], 5)
print(hampers)

[[0, 2], [0, 1], [0], [1, 2], [1, 2]]


There are some additional functions that I've written that I will just import as the logic is not important to the problem. These will convert the above representation of the hampers to a more fleshed out dictionary with item names and hamper value. The other will print this information in a table.

In [6]:
from hamper_problem import utils

hampers_dict = utils.make_hamper_output(hampers, item_lookup)
print(utils.make_hamper_table(hampers_dict))

  Hamper  Items                Value
--------  -----------------  -------
       0  Bleach, Coffee        2800
       1  Bleach, Chickpeas     2010
       2  Bleach                 710
       3  Chickpeas, Coffee     3390
       4  Chickpeas, Coffee     3390


### Step 1 - Initialize Population

The genetic algorithm will need an inital set of solutions to optimised. These will be created by randomly assigning items to hampers. It will be important to do this in a way that ensure the hampers that are created are valid (i.e. no duplicate items) otherwise there is the potential for many or all of the random solutions to be invalid which can significantly slow down the optimisation.

This is done by keeping track of the hampers that an item can be added to (i.e. hampers that don't already contain that item).

In [15]:
def make_random_solution(num_hampers: int, item_lookup: dict) -> list:
    solutions = []
    for item in item_lookup:
        num_units = item_lookup[item]["total units"]
        pack = randomly_distribute_pack(num_units, num_hampers)
        solutions.append(pack)

    return solutions


def randomly_distribute_pack(num_units: int, num_hampers: int) -> list:
    # Will track which hampers an item can be assigned to.
    hamper_options = list(range(1, num_hampers + 1))
    
    import pdb; pdb.set_trace()

    # Go through each unit of an item and distribute it to a hamper
    item_assignments = []
    for i in range(num_units):
        # Randomly select a hamper from the available hampers to assign the item to
        available_hampers = len(hamper_options) - 1
        hamper_idx = randint(0, available_hampers)

        # Remove the hamper as it is no longer available
        hamper = hamper_options.pop(hamper_idx)

        item_assignments.append(hamper)

    return item_assignments

The function above creates a single solution. We'll now write a function that creates a population of request size.

In [ ]:
def initialise_population(
    pop_size: int,
    num_hampers: int,
    item_lookup: dict
) -> list:
    population = []

    # Iterate through chromosomes in the population and get a randomised solution
    for i in range(pop_size):
        solution = make_random_solution(num_hampers, item_lookup)
        population.append(solution)

    return population

# Population of 2 for example
initialise_population(1, 2, item_lookup)

> c:\users\jackb\appdata\local\temp\ipykernel_9020\2635907463.py(18)randomly_distribute_pack()

ipdb> hamper_options
[1, 2]
ipdb> randint(0, available_hamper)
*** NameError: name 'available_hamper' is not defined
ipdb> randint(0, 2)
0
ipdb> randint(0, 1)
1
ipdb> randint(0, 0)
0


We'll now used the functions above to initialise our population. We will start with 25 hampers since that should be the ideal number based on our initial predictions. The number of solution in our population is set to 100 this is pretty arbitrary and probably overkill.

In [9]:
# Setting how large our population is
pop_size = 100

# Make the initial population
initial_population = initialise_population(pop_size, num_hampers, item_lookup)

print(initial_population[0])

[[6, 4, 5, 15, 12, 14, 13, 25, 18, 22], [1, 23, 14, 6, 21, 12, 9, 19, 24, 20], [2, 11, 19, 16, 18, 22, 8, 21, 20, 14], [18, 17, 21, 11, 14, 2, 20, 22, 5, 15], [11, 24, 19, 14, 10, 20, 7, 1, 16, 6], [2, 1, 22, 11, 7, 8, 12, 3, 21, 15], [4, 22, 24, 14, 6, 20, 25, 8, 18], [24, 23, 22, 13, 19, 21, 2, 25, 1, 12, 9, 10], [18, 2, 1, 3, 14, 20, 25, 16, 15, 4], [4, 12, 6, 24, 19, 9, 25, 14, 8, 5], [1, 12, 11, 19, 5, 17, 7, 13, 20, 10], [25, 23, 12, 17, 1, 22, 20, 10, 15, 21], [13, 21, 18, 25, 4, 10, 5, 3, 24, 20], [19, 4, 16, 12, 21, 14, 9, 1, 5, 6, 3, 17], [10, 21, 8, 13, 18, 11, 2, 3, 22, 17, 16, 20]]


### Step 2 - Fitness function
Once we have an initial population to work with, we need a way of assessing the fitness of a given solution. 
We then need to assign a score based on the sum absolute difference of the hampers from the target value of 5000. To do this will we calculate the ideal sum value of the hampers $(5000 \times n_{hampers})$. Our fitness score will be based on how far from this we are. So we will be aiming to maximise the equation below.

$$ 5000 \times n_{hampers} - \sum_{i=1}^{n_{hampers}} | 5000 - \sum_{i=1}^{n_{i}} price_i | $$

Solutions that are invalid are given a fitness of 0.

In [ ]:
def calc_fitness(
    chromosome: list,
    item_lookup: dict,
    num_hampers: int,
    target_hamper_value: float
) -> float:
    # Make hampers from chromosome so we can calculate their value
    hampers = chromosome_to_hampers(chromosome, num_hampers)

    # Hamper value for all hampers in solution so we can find the sum of the absolute
    hamper_values = [calc_hamper_value(hamper, item_lookup) for hamper in hampers]

    # difference from 5000 across all hampers
    diffs = [abs(value - target_hamper_value) for value in hamper_values]

    return sum(diffs)

### 4.4 Selection Function
Now we have a function for determining fitness of a solution we now need to write a function that selects solutions to use a parents for the next generation.

In [ ]:
def selection(fitness: list, num_parents: int, population: list) -> list:
    """Select the fittest solutions to use as parents for the next generation."""
    # Predefine array of indivduals that will be parents for the next generation
    parents = np.empty((num_parents, population.shape[1]))
    for i in range(num_parents):
        # Finding the fittest solution
        max_fitness_idx = np.where(fitness == np.max(fitness))
        parents[i,:] = population[max_fitness_idx[0][0], :]
        # Ensuring this solution is not selected next time
        fitness[max_fitness_idx[0][0]] = -999999
    return parents

### 4.5 Crossover Function
Here we perform crossover (aka getting it on) to create a new bunch of solutions. This is using two-point crossover. Crossover rate is high to ensure a large number of the fittest individuals are used.

Crossover works something like this
```
[0,0,0,0,0,0,0,0,0] Parent 1
     crossover     
[1,1,1,1,1,1,1,1,1] Parent 2
         =         
[0,0,0,1,1,1,0,0,0] Offspring 1

[1,1,1,0,0,0,1,1,1] Offspring 2
```

In [ ]:
def crossover(parents, num_offspring, crossoverRate=0.8):
    # Predefine the array for storing our offspring
    offspring = np.empty((numOffspring, parents.shape[1]))
    
    # Where to split the solutions is randomised
    crossoverPoint = randint(0, parents.shape[1]-1)
    
    for i in range(0, numOffspring, 2):
        # Get two parents and have them get it on
        parent1_index = i
        parent2_index = i+1
        
        # Skipping these parents if the crossover rate is exceeded
        x = rd.random()
        if x > crossoverRate:
            offspring[i, :] = parents[parent1_index, :]
            offspring[i+1, :] = parents[parent2_index, :]
            continue
               
        # Babies are made
        offspring[i,0:crossoverPoint]  = parents[parent1_index,0:crossoverPoint]
        offspring[i,crossoverPoint:]   = parents[parent2_index,crossoverPoint:]
        
        offspring[i+1,0:crossoverPoint]  = parents[parent2_index,0:crossoverPoint]
        offspring[i+1,crossoverPoint:]   = parents[parent1_index,crossoverPoint:]

    return offspring

### 4.6 Mutation Function
We will randomly mutate a few chromosomes (solutions) in order to see if we get any superpowers. We'll use the bit flip technique for creating mutants i.e. changing 1 to 0 or vice-versa.

In [ ]:
def mutation(offsprings, numHampers, itemIds, mutation_rate=0.4):
    # Predefine the mutants array
    mutants = np.empty((offsprings.shape))
    
    for i in range(mutants.shape[0]):
        random_value = rd.random()
        mutants[i,:] = offsprings[i,:]
        
        # Skip this one if our value exceeds the mutation rate
        if random_value > mutation_rate:
            continue
        
        # Randomise one gene in the chromosome
        idx1 = randint(0,offsprings.shape[1]-1)
        idx2 = randint(0,offsprings.shape[1]-1)
        val1 = mutants[i, idx1].copy()
        val2 = mutants[i, idx2].copy()
        
        mutants[i, idx1] = val2
        mutants[i, idx2] = val1

    return mutants   

### 4.7 Termination Function
This function will check if the optimisation should be terminated. The optimisation will be terminated if a certain number of generations have passed with no change in max fitness. It will also be terminated if a target fitness it met or exceeded.

In [ ]:
def termination(fitnessHistory, tolerance, targetFitness):
    fitnessHistory_max = [np.max(fitness) for fitness in fitnessHistory[-tolerance:]]
    fitnessHistory_max = np.array(fitnessHistory_max)
    
    if len(fitnessHistory) < tolerance:
        return False
    elif np.any(fitnessHistory_max >= targetFitness):
        print("Terminated Early: Target fitness equalled or exceeded")
        return True
    elif np.all(fitnessHistory_max == fitnessHistory_max[-1]):
        print(f"Terminated Early: Max fitness has not changed in {tolerance} generations")
        return True
    else:
        return False

### 4.8 Bring it together
We'll bring all of these functions together to create the Genetic Algorithm optimisation process.

In [ ]:
def optimize(itemIds, itemNames, prices, numHampers, population, popSize, numGenerations, tolerance, targetFitness):
    parameters, fitnessHistory = [], []
    numParents = int(popSize[0]/2)
    numOffspring = popSize[0] - numParents 
    
    # Iterate to optimise for the selected number of generations
    for i in range(numGenerations):
        # Determine the fittness of our the individuals in the populations
        fitness = calcFitness(itemNames, prices, numHampers, population)

        # Save fitness history so we can plot etc later
        fitnessHistory.append(fitness)
        
        # See if we have satified a termination condition
        terminate = termination(fitnessHistory, tolerance, targetFitness)
        
        # Terminate the optimisation early
        if terminate:
             # Find the fittest individual in the final generation
            fittestSolution = np.where(fitness == np.max(fitness))
            # Get the fittest solution for display
            parameters.append(population[fittestSolution[0][0],:])
            return parameters, population, fitnessHistory

        # Select parents for getting it on
        parents = selection(fitness, numParents, population)

        # Make babbies
        offspring = crossover(parents, numOffspring)

        # Teenage mutant ninja turtles
        # Some are mutated some remain the same
        mutants = mutation(offspring, numHampers, itemIds)
        
        # New populations
        population[0:parents.shape[0], :] = parents
        population[parents.shape[0]:, :] = mutants

    # Determine fittness of the final generation
    fitness = calcFitness(itemNames, prices, numHampers, population)   
    
    # Find the fittest individual in the final generation
    fittestSolution = np.where(fitness == np.max(fitness))

    # Get the fittest solution for display
    parameters.append(population[fittestSolution[0][0],:])
    
    return parameters, population, fitnessHistory

## 5 Running the Genetic Algorithm
To run this optimisation we need to first create a random set of solutions

In [ ]:
%%time
numGenerations = 1000

targetFitness = 5000*numHampers - 1709

parameters, population, fitnessHistory = optimize(
    itemIds,
    itemNames,
    prices,
    numHampers,
    initialPopulation,
    popSize,
    numGenerations,
    200,
    targetFitness
)


In [ ]:
hampers_dict = chromosomeToHampers(parameters[0], itemNames, prices, numHampers)
print("========== Hampers ==========")
for key in hampers_dict:
    print(f'{hampers_dict[key]["items"]};{np.sum(hampers_dict[key]["prices"])}')
    
print("\nSum of abs diff from 5000 = ", 5000*numHampers - np.max(fitnessHistory))

In [ ]:
# Calculate the mean and max fitness values for each generation
fitnessHistory_mean = [np.mean(fitness) for fitness in fitnessHistory]
fitnessHistory_max = [np.max(fitness) for fitness in fitnessHistory]

# Plot fitness through the generations
plt.plot(list(range(len(fitnessHistory))), fitnessHistory_mean, label = 'Mean Fitness')
plt.plot(list(range(len(fitnessHistory))), fitnessHistory_max, label = 'Max Fitness')
plt.legend()
plt.title('Fitness through the generations')
plt.xlabel('Generations')
plt.ylabel('Fitness')
plt.show()
print(np.asarray(fitnessHistory).shape)